In [1]:
import pyodbc
import os
import time
import csv
 
def sqlCursor():
    sql_driver = 'DRIVER={ODBC Driver 13 for SQL Server};'
    sql_server = 'SERVER=sage;'
    sql_db = 'DATABASE=BUYS;'
    sql_UID = 'Trusted_Connection=yes;'

    cnxn = pyodbc.connect(sql_driver + sql_server + sql_db + sql_UID)
    cursor = cnxn.cursor()
    return cursor
 
# def writeFile(data, fn):
#     with open(fn,'w',encoding='utf-8') as file:
#         file.write('ASIN,Thicket,isUsed,Price,AmazonSalesRank,AmazonLowestPrice,PricedAtFloor,CostOfGoods,DaysInStock\n')
#         for x in data:
#             file.write(str(x.ASIN) + ',' + 
#                        str(x.Thicket) + ',' + 
#                        str(x.isUsed) + ',' +
#                        str(x.Price) + ',' +
#                        str(x.AmazonSalesRank) + ',' +
#                        str(x.AmazonLowestPrice) + ',' +
#                        str(x.PricedAtFloor) + ',' +
#                        str(x.CostOfGoods) + ',' +
#                        str(x.DaysInStock) + 
#                        '\n')
            
            
def writeCSV(fn, data):
    with open(fn, 'w', newline='') as outcsv:
        writer = csv.writer(outcsv, delimiter=',', quoting=csv.QUOTE_MINIMAL)
       # writer.writerow(['ASIN', 'Thicket', 'isUsed', 'Price', 'AmazonSalesRank', 'AmazonLowestPrice', 'PricedAtFloor', 'CostOfGoods', 'DaysInStock'])
        for row in data:
            writer.writerow(row)

def fetchData(sql, query):
    iter = sql.execute(query)
    yield [column[0] for column in iter.description]
    while True:
        rows = iter.fetchmany(50000)
        if not rows:
            break
        for row in rows:
            yield row
            
def readQueryFile(fn):
    # Open and read the file as a single buffer
    f = open(fn, 'r')
    query = f.read()
    f.close()
    return query

def main():
    query = f'''
    SET NOCOUNT ON
    SET ANSI_WARNINGS OFF

    SELECT 
        [ASIN],
        RIGHT(li.ServerName, 1) [Thicket],
        CASE WHEN SKU LIKE 'S_%' THEN 1 ELSE 0 END [isUsed],
        Price,
        CASE WHEN im.AmazonSalesRank = 0 THEN NULL ELSE im.AmazonSalesRank END [AmazonSalesRank],
        im.AmazonLowestPrice,
        CASE WHEN PricedAtFloor = 'True' THEN 1 ELSE 0 END [PricedAtFloor],
        CostOfGoods,
        DATEDIFF(DAY, InsertDate, GETDATE()) [DaysInStock]
    FROM [ISIS].[dbo].[Inventory_Monsoon] im
        INNER JOIN ISIS.dbo.App_ListingInstances li
            ON im.ListingInstanceID = li.ListingInstanceID
            AND li.[Status] = 'A'
    WHERE RIGHT(li.ServerName, 1) <> '7'
    '''
    sql = sqlCursor()
    data =fetchData(sql, query)
    writeCSV('./data/iStoreInventoryDetail_test.csv', data)
 
if __name__ == '__main__':
    t0 = time.time()
    main()
    t1 = time.time()
    total = t1-t0
    print(total)

101.31623339653015


In [2]:
def readQueryFile(fn):
    # Open and read the file as a single buffer
    f = open(fn, 'r')
    query = f.read()
    f.close()
    return query